This program will read in the designed GTFS xml format file and combine all trip departure tags with the same shape ID

In [2]:
import xml.etree.ElementTree as elem
# Get the root of this tree as schedule_root
schedule_root = elem.parse('BK-schedule.xml').getroot()

In [3]:
# Find the tag for schedule_root
schedule_root.tag

'transitSchedule'

In [4]:
# Then we could see all the index desired in the index array
index_array = []
for child in schedule_root:
    index = child.get('id')
    index_array.append(index)
print(index_array)

[None, 'B11_B11', 'B12_B12', 'B13_B13', 'B14_B14', 'B15_B15', 'B16_B16', 'B17_B17', 'B1_B1', 'B20_B20', 'B24_B24', 'B25_B25', 'B26_B26', 'B2_B2', 'B31_B31', 'B32_B32', 'B35_B35', 'B36_B36', 'B37_B37', 'B38_B38', 'B39_B39', 'B3_B3', 'B41_B41', 'B42_B42', 'B43_B43', 'B44-SBS_B44+', 'B44_B44', 'B45_B45', 'B46_B46', 'B47_B47', 'B48_B48', 'B49_B49', 'B4_B4', 'B52_B52', 'B54_B54', 'B57_B57', 'B60_B60', 'B61_B61', 'B62_B62', 'B63_B63', 'B64_B64', 'B65_B65', 'B67_B67', 'B68_B68', 'B69_B69', 'B6_B6', 'B70_B70', 'B74_B74', 'B7_B7', 'B82_B82', 'B83_B83', 'B84_B84', 'B8_B8', 'B9_B9', 'Bx10_BX10', 'Bx11_BX11', 'Bx12-SBS_BX12+', 'Bx12_BX12', 'Bx13_BX13', 'Bx15_BX15', 'Bx16_BX16', 'Bx17_BX17', 'Bx18_BX18', 'Bx19_BX19', 'Bx1_BX1', 'Bx20_BX20', 'Bx21_BX21', 'Bx22_BX22', 'Bx24_BX24', 'Bx26_BX26', 'Bx27_BX27', 'Bx28_BX28', 'Bx29_BX29', 'Bx2_BX2', 'Bx30_BX30', 'Bx31_BX31', 'Bx32_BX32', 'Bx33_BX33', 'Bx34_BX34', 'Bx35_BX35', 'Bx36_BX36', 'Bx38_BX38', 'Bx39_BX39', 'Bx3_BX3', 'Bx40_BX40', 'Bx41-SBS_BX41+', '

In [5]:
# Make a copy of the root:
root = schedule_root
# Initialize a dictionary to store all route profile corresponding to the shapeID:
shapeID_dict = {}
# Then we try to figure out the index interested
index_interested = []
for child in root:
    # First we have to find all the transitLine children:
    if child.tag == 'transitLine':
        # Second we have to find out the transitLine which we are interested in:
        if child.getchildren() != []:
            # Iterate through all this transitLine elemenst interested
            for transitRoute_child in child:
                # Find the description tag including shapeID
                description_child = transitRoute_child.find('description')
                ID = description_child.text[8:]
                routeProfile = transitRoute_child.find('routeProfile')
                # And we filter out those do have a shapeID description
                if ID not in shapeID_dict:
                    # If the shapeID is new, we should add to the dictionary: 
                    elem_dict = []
                    for item in routeProfile:
                        elem_dict.append(item)
                    shapeID_dict[ID] = elem_dict
                else:
                    # If this ID already in the dictionary, we just update/append the new routeProfile:
                    current_content = shapeID_dict.get(ID)
                    for item in routeProfile:
                        current_content.append(item)
                    shapeID_dict[ID] = current_content

# print(shapeID_dict.get('B1O0264')) 

In [6]:
# Then we could construct the table according to the description, i.e. shapeID, we have:
key = 'B1O0264' # Change this line
stop_list = shapeID_dict.get(key)
# We should remove the first stop element as it is the offset, and we could confirm that:
departure = stop_list.pop(0)
departure_refId = departure.get('refId')
print('The departure refID is '+ departure_refId)
departure_offset = departure.get('departureOffset')
print('The departure offset is '+ departure_offset)
await_departure = departure.get('awaitDeparture')
print('await departure? '+ await_departure)

# Warning: Do not run this block alone more than once since it modifies the list structure

The departure refID is 901473
The departure offset is 00:00:00
await departure? true


In [10]:
from astropy.table import Table, Column
import numpy as np
for element_stop in stop_list:
    t = Table(names=('refID', 'arrivalOffset', 'departureOffset','awaitDeparture'),dtype='string')
    t.add_row([element_stop.get('refID'), element_stop.get('arrivalOffset'), element_stop.get('departureOffset'), element_stop.get('awaitDeparture')])

ValueError: Arguments "names" and "dtype" must match number of columns